In [1]:
# 패키지 가져오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rn
import seaborn as sns


In [2]:
# 데이터 읽기
train = pd.read_csv("train.csv",header=None)
test = pd.read_csv("test.csv",header=None)
train_label = pd.read_csv("trainLabels.csv",header=None)
print('Train data size : ', train.shape)
print('Test data size : ', test.shape)
print('Train label data size : ', train_label.shape)

Train data size :  (1000, 40)
Test data size :  (9000, 40)
Train label data size :  (1000, 1)


In [3]:
# train 데이터와 validation 데이터로 분리
from sklearn.model_selection import train_test_split
train_label = np.ravel(train_label)
Xtrain, Xval, ytrain, yval = train_test_split(train, train_label, random_state=1, test_size=0.2)
print(Xtrain.shape)
print(Xval.shape)
print(ytrain.shape)
print(yval.shape)

(800, 40)
(200, 40)
(800,)
(200,)


In [4]:
pip install xgboost

In [5]:
pip install lightgbm

In [6]:
# 사이킷런 분류알고리즘, 클래스 불러오기
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import numpy as npa

In [7]:
#XGBOOST
xgb = XGBClassifier()
xgb.fit(Xtrain, ytrain)
xgb_predicted = xgb.predict(Xval)
print('XGBoost:', accuracy_score(yval, xgb_predicted))

#SVC
svc_model = SVC(gamma = 'auto') # gamma = 'auto': 커널 계수 자동 조정
svc_model.fit(Xtrain, ytrain)
svc_predicted = svc_model.predict(Xval)
print('SVM:', accuracy_score(yval, svc_predicted))

XGBoost: 0.93
SVM: 0.915


In [11]:
# GridSearch 가 찾을 Parameter를 정의
xgb_param_grid={
    'n_estimators' : [300],
    'learning_rate' : [0.05],
    'max_depth' : [5],
    'gamma' : [0],
    'colsample_bytree' : [0.9],

}

svc_param_grid = {'C':[1],'gamma':[0.1], 'kernel':['linear']}

In [12]:
# cross validation 지정
from sklearn.model_selection import KFold

kf = KFold(random_state=30,
           n_splits=10,
           shuffle=True,
          )

In [13]:
# xbg 적용
xgb_grid=GridSearchCV(xgb, param_grid = xgb_param_grid, scoring="f1_macro", n_jobs=-1, verbose = 2)
xgb_grid.fit(Xtrain, ytrain)


Fitting 5 folds for each of 1 candidates, totalling 5 fits


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_b...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.9], 'gamma': [0],
                         'learning_rate': [0.05], 'max_depth': [5],
                         'n_estimators': [300]},
             scoring='f1_macro', verbose=2)

In [14]:
# 그리스서치 xgb 최적의 파라미터 확인
xgb_grid.best_params_

{'colsample_bytree': 0.9,
 'gamma': 0,
 'learning_rate': 0.05,
 'max_depth': 5,
 'n_estimators': 300}

In [15]:
# svc 적용
svc_grid=GridSearchCV(xgb, param_grid = svc_param_grid, scoring="f1_macro", n_jobs=-1, verbose = 2)
svc_grid.fit(Xtrain, ytrain)


Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [16:09:29] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "C", "kernel" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_b...ne,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'C': [1], 'gamma': [0.1], 'kernel': ['linear']},
             scoring='f1_macro', verbose=2)

In [16]:
# 그리드서치 svc 최적의 파라미터 확인
svc_grid.best_params_

{'C': 1, 'gamma': 0.1, 'kernel': 'linear'}

In [17]:
# 그리드서치로 찾은 파라미터 적용해서 xgb 성능 비교하기~~~~~
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

xgb_grid_predicted = xgb_grid.predict(Xval)

print(classification_report(yval, xgb_grid_predicted))
print(confusion_matrix(yval, xgb_grid_predicted))


              precision    recall  f1-score   support

           0       0.94      0.93      0.93        95
           1       0.93      0.94      0.94       105

    accuracy                           0.94       200
   macro avg       0.94      0.93      0.93       200
weighted avg       0.94      0.94      0.93       200

[[88  7]
 [ 6 99]]


In [20]:
# 그리드서치로 찾은 파라미터 적용해서 svc 성능 비교하기!!!!!!
svc_grid_predicted = svc_grid.predict(Xval)

print(classification_report(yval, svc_grid_predicted))
print(confusion_matrix(yval, svc_grid_predicted))

              precision    recall  f1-score   support

           0       0.93      0.92      0.92        95
           1       0.92      0.93      0.93       105

    accuracy                           0.93       200
   macro avg       0.93      0.92      0.92       200
weighted avg       0.93      0.93      0.92       200

[[87  8]
 [ 7 98]]


In [21]:
test = pd.read_csv('test.csv', header=None)

In [24]:
xgb_grid_predicted = xgb_grid.predict(test)
svc_grid_predicted = svc_grid.predict(test)

xgb_results = pd.DataFrame(xgb_grid_predicted, columns=['Solution'])
xgb_results.index = xgb_results.index + 1
xgb_results.index.name = "Id"


svc_results = pd.DataFrame(svc_grid_predicted, columns=['Solution'])
svc_results.index = svc_results.index + 1
svc_results.index.name = "Id"

In [29]:

xgb_results.to_csv('xgb_predictions_GS.csv', index=True)
svc_results.to_csv('svc_predictions_GS.csv', index=True)


In [28]:
svc_results

,Solution
Id,
1,1
2,0
3,0
4,0
5,0
...,...
8996,1
8997,1
8998,1
